In [1]:

import itertools
import pandas as pd
import cobra.flux_analysis
from cobra import Metabolite, Reaction, Model
import time
import numpy as np
from functools import partial
from src.mp_functions import combinations_subset, parallelize_dataframe, knockout_FBA, knockout_FBA_w_tasks

from functools import partial
from src.met_task_functions import get_met_ids, constrain_model, create_reactions


"""A mess of a document with different code cells.
Good to to use for any testing that involves the Recon3D model as it takes some time to load in."""

start_time = time.time()
model_file_path = 'C:/Users/Sigve/Genome_Data/Human1/Human1_GEM/GTEx/brain.xml'
model = cobra.io.read_sbml_model(model_file_path)


end_time = time.time()
print('Model load time: %.6f seconds' % (end_time - start_time))

Academic license - for non-commercial use only - expires 2022-10-03
Using license file c:\gurobi\gurobi.lic
Model load time: 30.027146 seconds


In [22]:
t_model = model.copy()
#t_model.name = 't_model'
u_model = model.copy()
print(model.name)
print(t_model.name)
print(u_model.name)

Read LP format model from file C:\Users\Sigve\AppData\Local\Temp\tmplzeolpto.lp
Reading time = 0.07 seconds
: 5482 rows, 15138 columns, 66608 nonzeros
Read LP format model from file C:\Users\Sigve\AppData\Local\Temp\tmpzj5b9nmc.lp
Reading time = 0.07 seconds
: 5482 rows, 15138 columns, 66608 nonzeros
Model Exported from COBRA Toolbox
Model Exported from COBRA Toolbox
Model Exported from COBRA Toolbox


In [23]:
for rx in t_model.exchanges:
    m = list(rx.metabolites.keys())[0]
    boundary_met = Metabolite(m.id[:-4] + 'x[x]', formula=m.formula, name=' '.join(m.name.split(' ')[:-1]) + ' [Boundary]', compartment='x')
    rx.add_metabolites({boundary_met: 1})

print(len(t_model.exchanges))

There are no boundary reactions in this model. Therefore, specific types of boundary reactions such as 'exchanges', 'demands' or 'sinks' cannot be identified.


0


In [24]:
print(t_model.name)
print(len(t_model.metabolites))
print(len(model.metabolites))
print(len(u_model.metabolites))

Model Exported from COBRA Toolbox
6348
5482
5482


In [33]:
t = t_model.metabolites.get_by_id('m02560x[x]')
print(t.formula)
print(t.name)
print(t.reactions)

r1 = model.reactions.get_by_id('HMR_0015')
r2 = model.reactions.get_by_id('HMR_9033')
print(r1)
print(r2)

CHO2R
NEFA blood pool in [Boundary]
frozenset({<Reaction HMR_9033 at 0x253014a1b80>})
HMR_0015: m02560s[s] --> 0.0048 m00003s[s] + 0.0016 m00008s[s] + 0.0016 m00010s[s] + 0.0001 m00017s[s] + 0.0001 m00019s[s] + 0.0001 m00021s[s] + 0.0001 m00094s[s] + 0.0001 m00104s[s] + 0.0001 m00111s[s] + 0.0001 m00114s[s] + 0.0001 m00115s[s] + 0.0001 m00117s[s] + 0.0001 m00128s[s] + 0.0001 m00132s[s] + 0.0001 m00135s[s] + 0.0001 m00260s[s] + 0.0001 m00265s[s] + 0.0001 m00315s[s] + 0.0001 m00341s[s] + 0.0001 m01197s[s] + 0.0001 m01207s[s] + 0.0001 m01235s[s] + 0.0001 m01238s[s] + 0.0017 m01291s[s] + 0.0137 m01362s[s] + 0.0007 m01373s[s] + 0.0005 m01432s[s] + 0.0001 m01582s[s] + 0.0002 m01583s[s] + 0.0001 m01584s[s] + 0.0001 m01585s[s] + 0.0046 m01689s[s] + 0.0025 m01696s[s] + 0.0019 m01741s[s] + 0.0011 m01771s[s] + 0.0001 m01778s[s] + 0.002 m01784s[s] + 0.0048 m01932s[s] + 0.0001 m02053s[s] + 0.0034 m02344s[s] + 0.0012 m02385s[s] + 0.071 m02387s[s] + 0.0005 m02389s[s] + 0.0056 m02456s[s] + 0.0005 m024

In [173]:
# Import table
tasks = pd.read_table('C:/Users/Sigve/Genome_Data/Human1/Human1_GEM/tasks/essential_tasks.tsv')

print(tasks.head())

   id                                    description  shouldFail  printFluxes  \
0  ER  Aerobic rephosphorylation of ATP from glucose           0            0   
1  ER               Aerobic rephosphorylation of GTP           0            0   
2  ER               Aerobic rephosphorylation of CTP           0            0   
3  ER               Aerobic rephosphorylation of UTP           0            0   
4  BS                          ATP de novo synthesis           0            0   

   comments                        inputs     LBin                 UBin  \
0       NaN             'O2[s]glucose[s]'      0,0            1000,1000   
1       NaN             'O2[s]glucose[s]'      0,0            1000,1000   
2       NaN             'O2[s]glucose[s]'      0,0            1000,1000   
3       NaN             'O2[s]glucose[s]'      0,0            1000,1000   
4       NaN  'O2[s]glucose[s]NH3[s]Pi[s]'  0,0,0,0  1000,1000,1000,1000   

                outputs  LBout           UBout  \
0        'H2

In [174]:
# Formating data
tasks['LBin'] = tasks['LBin'].apply(lambda x: x.split(','))
tasks['LBout'] = tasks['LBout'].apply(lambda x: x.split(','))
tasks['UBin'] = tasks['UBin'].apply(lambda x: x.split(','))
tasks['UBout'] = tasks['UBout'].apply(lambda x: x.split(','))

d = ']'
tasks['inputs'] = tasks['inputs'].apply(lambda x: [e+d for e in x[1:-1].split(d)][0:-1])
tasks['outputs'] = tasks['outputs'].apply(lambda x: [e+d for e in x[1:-1].split(d)][0:-1])
tasks['equations'] = tasks['equations'].apply(str)
print(tasks.head())

   id                                    description  shouldFail  printFluxes  \
0  ER  Aerobic rephosphorylation of ATP from glucose           0            0   
1  ER               Aerobic rephosphorylation of GTP           0            0   
2  ER               Aerobic rephosphorylation of CTP           0            0   
3  ER               Aerobic rephosphorylation of UTP           0            0   
4  BS                          ATP de novo synthesis           0            0   

   comments                              inputs          LBin  \
0       NaN                 [O2[s], glucose[s]]        [0, 0]   
1       NaN                 [O2[s], glucose[s]]        [0, 0]   
2       NaN                 [O2[s], glucose[s]]        [0, 0]   
3       NaN                 [O2[s], glucose[s]]        [0, 0]   
4       NaN  [O2[s], glucose[s], NH3[s], Pi[s]]  [0, 0, 0, 0]   

                       UBin                   outputs      LBout  \
0              [1000, 1000]          [H2O[s], CO2[s]] 

In [81]:
comps = {'s': 'Extracellular',
'p': 'Peroxisome',
'm': 'Mitochondria',
'c': 'Cytosol',
'l': 'Lysosome',
'r': 'Endoplasmic reticulum',
'g': 'Golgi apparatus',
'n': 'Nucleus',
'i': 'Inner mitochondria',
'x': 'Boundary'}

In [175]:
met_list = []

for i, data in tasks.iterrows():
    l = []
    if data['equations'] != 'nan':
        l = data['equations'].split(' ')
        l = [item for item in l if item[-1] == ']']

    met_list = met_list + data['inputs'] + data['outputs'] + l


met_list = list(set(met_list))
print(len(met_list))


120


In [176]:
mets = t_model.metabolites

met_ids = {}
for met in met_list:


    if met[:-3] == 'ALLMETSIN':
        met_ids[met] = 'ALLMETSIN'

    elif met[:-3] == 'ALLMETS':
        met_ids[met] = 'ALLMETS'

    else:
        comp = met[-2]
        temp = met[:-3] + ' [{0}]'.format(comps[comp])
        for m in mets:
            if m.name == temp:
                met_ids[met] = m
                break
        else:
            # Failed to find
            print(temp)
            print(met)


print(len(met_ids))

120


In [177]:
tasks.at[56, 'outputs']= [tasks.at[56, 'outputs'][1]] + tasks.at[56, 'inputs']
tasks.at[56, 'LBout']= [tasks.at[56, 'LBout'][1]] + tasks.at[56, 'LBin']
tasks.at[56, 'UBout']= [tasks.at[56, 'UBout'][1]] + tasks.at[56, 'UBin']

print(tasks.loc[56].at['outputs'])
print(len(tasks.loc[56].at['LBout']))
print(len(tasks.loc[56].at['UBout']))

['biomass[s]', 'arginine[s]', 'histidine[s]', 'lysine[s]', 'methionine[s]', 'phenylalanine[s]', 'tryptophan[s]', 'tyrosine[s]', 'alanine[s]', 'glycine[s]', 'serine[s]', 'threonine[s]', 'aspartate[s]', 'glutamate[s]', 'asparagine[s]', 'glutamine[s]', 'isoleucine[s]', 'leucine[s]', 'proline[s]', 'valine[s]', 'cysteine[s]', 'thiamin[s]', 'hypoxanthine[s]', 'folate[s]', 'biotin[s]', 'pantothenate[s]', 'choline[s]', 'inositol[s]', 'nicotinamide[s]', 'pyridoxine[s]', 'riboflavin[s]', 'thymidine[s]', 'aquacob(III)alamin[s]', 'lipoic acid[s]', 'glucose[s]', 'sulfate[s]', 'linoleate[s]', 'linolenate[s]', 'O2[s]', 'H2O[s]', 'retinoate[s]', 'Fe2+[s]', 'Pi[s]', 'alpha-tocopherol[s]', 'gamma-tocopherol[s]']
45
45


In [178]:
rx_list = []
in_list = []
out_list = []
for ind, data in tasks.iterrows():

    for i, name, ml, lbs, ubs in zip([1, -1], ['in', 'out'], [data.inputs, data.outputs], [data.LBin, data.LBout], [data.UBin, data.UBout]):
        rxl = []
        for j, m, lb, ub in zip( range(len(ml)),ml, lbs, ubs):
            rx = Reaction('ess_{0}_{1}_{2}'.format(ind+1, name, j))
            rx.add_metabolites({met_ids[m]: i})
            rx.lower_bound = float(lb)
            rx.upper_bound = float(ub)
            rxl.append(rx)
        if name == 'in':
            in_list.append(rxl)
        else:
            out_list.append(rxl)

    if data.equations != 'nan':
        t = [ [ met_ids[subsub] for subsub in sub.split(' ') if len(subsub)>1] for sub in data.equations.split('=') ]
        d = {}

        for i, ml in zip([-1, 1], t):
            for m in ml:
                d[m] = i

        rx = Reaction('ess_{0}'.format(ind+1))
        rx.add_metabolites(d)
        rx.lower_bound = float(data.LBequ)
        rx.upper_bound = float(data.UBequ)
        rx.name = data.description

        rx_list.append(rx)

    else:
        rx_list.append('nan')
task_list = pd.DataFrame(list(zip(in_list, out_list, rx_list)), columns=['in_rx', 'out_rx', 'equ'])

In [179]:
print(t_model.optimize())

<Solution 0.000 at 0x253a7d555e0>


In [226]:
n=0
sol = cobra.Solution
res = []
for ind, data in task_list.iterrows():

    n+=1
    if n>60:
        break
    #print(data.equ)


    with t_model:
        
        for rx in data.in_rx + data.out_rx:
            t_model.add_reaction(rx)

        if data.equ != 'nan':
            t_model.add_reaction(data.equ)

        for gene_id in ["ENSG00000165449", "ENSG00000100714"]:
            t_model.genes.get_by_id(gene_id).knock_out()

        sol = t_model.optimize()

    res.append(sol.objective_value)





C:\Users\Sigve\miniconda3\envs\Master\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


In [210]:
print(res[50])

None


In [225]:
print(sol)
print(task_list.iloc[36, 0])
print(task_list.iloc[36, 1])


res_2 = sol.fluxes

print(res_2[res_2 != 0])

<Solution infeasible at 0x25308513b50>
[<Reaction ess_37_in_0 at 0x253a7075f70>, <Reaction ess_37_in_1 at 0x25301258220>, <Reaction ess_37_in_2 at 0x253012580a0>]
[<Reaction ess_37_out_0 at 0x25301258580>, <Reaction ess_37_out_1 at 0x25301258280>, <Reaction ess_37_out_2 at 0x25301259bb0>, <Reaction ess_37_out_3 at 0x25301259610>]
Series([], Name: fluxes, dtype: float64)


In [2]:
model_list = constrain_model(model, ALLMETSIN=True)

Read LP format model from file C:\Users\Sigve\AppData\Local\Temp\tmpzci9f6tp.lp
Reading time = 0.07 seconds
: 5482 rows, 15138 columns, 66608 nonzeros
Read LP format model from file C:\Users\Sigve\AppData\Local\Temp\tmprqjoxsmr.lp
Reading time = 0.07 seconds
: 5482 rows, 15138 columns, 66608 nonzeros
Read LP format model from file C:\Users\Sigve\AppData\Local\Temp\tmpp1oy6755.lp
Reading time = 0.07 seconds
: 5482 rows, 15138 columns, 66608 nonzeros


In [3]:
tasks_df = pd.read_table('C:/Users/Sigve/Genome_Data/Human1/Human1_GEM/tasks/essential_tasks.tsv')
# Formatting data
for b in ['LBin', 'LBout', 'UBin', 'UBout']:
    tasks_df[b] = tasks_df[b].apply(lambda x: x.split(','))

for put in ['inputs', 'outputs']:
    tasks_df[put] = tasks_df[put].apply(lambda x: [e + ']' for e in x[1:-1].split(']')][0:-1])

tasks_df['equations'] = tasks_df['equations'].apply(str)

tasks_df[['met_ids', 'model_num']] = tasks_df.apply(partial(get_met_ids, model_list), axis=1, result_type ='expand')

tasks_df = create_reactions(tasks_df)



In [4]:
essential = pd.read_csv('C:/Users/Sigve/Genome_Data/results/model_tests/brain_model_essential.csv', index_col=0)

essential['pass/fail'] = essential.values.tolist()
essential = essential[['pass/fail']].reset_index()
print(essential.shape[0])


essential = essential[essential['pass/fail'].map(lambda x: any(task == 1 for task in x))]
essential['gene_ids'] = essential['Var1'].apply(lambda x: [x])
essential = pd.DataFrame(essential[['gene_ids', 'pass/fail']])
essential['pass/fail'] = essential['pass/fail'].apply(lambda x: [1 if i==0 else 0 for i in x])
essential.reset_index(inplace=True, drop=True)
print(essential.head())
print(essential.shape[0])

2451
            gene_ids                                          pass/fail
0  [ENSG00000001630]  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
1  [ENSG00000005469]  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
2  [ENSG00000010256]  [1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, ...
3  [ENSG00000031698]  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
4  [ENSG00000048392]  [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, ...
174


In [5]:
print(essential.iloc[1,1])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0]


In [6]:
combinations_list = []
n=10
for g, df in essential.groupby(np.arange(len(essential)) // n):
    combinations_list.append(df)
print(combinations_list[0].shape[0])
print(len(combinations_list))

10
18


In [8]:
res_list=[]
start_time = time.time()
num = 1
for sub_df in combinations_list:
    temp_df = parallelize_dataframe(sub_df, partial(combinations_subset, partial(knockout_FBA_w_tasks, tasks_df, model_list)), 9)
    temp_df.to_csv(path_or_buf='C:/Users/Sigve/Genome_Data/results/results_FBA/result_{0}.tsv'.format(num), sep='\t')
    res_list.append(temp_df)
    num += 1

end_time = time.time()
print('FBA run time: %.6f seconds' % (end_time - start_time))

FBA run time: 1469.011253 seconds


In [10]:
result_df = pd.concat(res_list)

174
            gene_ids                                          pass/fail  \
0  [ENSG00000001630]  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
1  [ENSG00000005469]  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
2  [ENSG00000010256]  [1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, ...   
3  [ENSG00000031698]  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
4  [ENSG00000048392]  [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, ...   

                                             results   main_obj  \
0  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   0.000000   
1  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...  86.141528   
2  [1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, ...  85.302010   
3  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   0.000000   
4  [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, ...  87.693483   

                                          comparison  
0  [False, False, False, False, False, False, Fal...  
1  [False, False, False, False, 

In [18]:
start_time = time.time()
temp_df = parallelize_dataframe(combinations_list[0], partial(combinations_subset, partial(knockout_FBA_w_tasks, tasks_df, model_list)), 4)
end_time = time.time()
print('FBA run time: %.6f seconds' % (end_time - start_time))

FBA run time: 171.471487 seconds


In [19]:
print(temp_df)

            gene_ids                                          pass/fail  \
0  [ENSG00000001630]  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
1  [ENSG00000005469]  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
2  [ENSG00000010256]  [1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, ...   
3  [ENSG00000031698]  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
4  [ENSG00000048392]  [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, ...   
5  [ENSG00000060971]  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
6  [ENSG00000062485]  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
7  [ENSG00000065427]  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
8  [ENSG00000066926]  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
9  [ENSG00000067064]  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   

                                             results  
0  [0.0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1...  
1  [86.14152818507019, 1, 1, 1, 1, 1, 1, 1, 1, 1,...  
2  [85.30

In [20]:
result_df = temp_df.copy()

print(result_df.shape[0])


result_df['main_obj'] = result_df['results'].apply(lambda x: x[0])
result_df['results'] = result_df['results'].apply(lambda x: x[1:])

result_df['comparison'] = result_df[['pass/fail', 'results']].apply(lambda x: [False if i == j else True for i, j in zip(x[0], x[1])], axis=1)
print(result_df.head())

10
            gene_ids                                          pass/fail  \
0  [ENSG00000001630]  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
1  [ENSG00000005469]  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
2  [ENSG00000010256]  [1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, ...   
3  [ENSG00000031698]  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
4  [ENSG00000048392]  [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, ...   

                                             results   main_obj  \
0  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   0.000000   
1  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...  86.141528   
2  [1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, ...  85.302010   
3  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   0.000000   
4  [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, ...  87.693483   

                                          comparison  
0  [False, False, False, False, False, False, Fal...  
1  [False, False, False, False, F

In [21]:
indicies = []
for ind, data in result_df.iterrows():
    indicies.append([i for i, x in enumerate(data.comparison) if x])
print(indicies)

[[], [56], [56], [], [], [], [], [], [], []]


In [23]:
c1 = result_df.iloc[1,1]
c2 = result_df.iloc[1,2]
comp_df = pd.DataFrame(list(zip(['Fail' if 0==i else 'Pass' for i in c1], ['Fail' if 0==i else 'Pass' for i in c2])), columns=['MATLAB:', 'Python:'])
print(comp_df.iloc[56,:])

MATLAB:    Fail
Python:    Pass
Name: 56, dtype: object


In [22]:
test_df = result_df[result_df['comparison'].map(lambda x: any(x))]
test_df = test_df[['gene_ids', 'main_obj']].reset_index(drop=True)
print(test_df)


            gene_ids   main_obj
0  [ENSG00000005469]  86.141528
1  [ENSG00000010256]  85.302010


In [45]:
res = []
data = tasks_df.iloc[56, :]
for ind, data2 in test_df.iterrows():

    if ind > 0:
        break

    t_model = model_list[data.model_num]

    with t_model:
        for subset in [data.in_rx, data.out_rx]:
            for rx in subset:
                if rx == 'ALLMETSIN':
                    # Adds boundary metabolites for other reactions when ALLMETSIN is used
                    for r in subset[1:]:
                        for m2 in r.metabolites:
                            for r2 in m2.reactions:
                                if r2.boundary and r2.id != r.id:
                                    r2.add_metabolites({Metabolite(
                                                        m2.id[:-4] + 'x[x]',
                                                        formula=m2.formula,
                                                        name=' '.join(m2.name.split(' ')[:-1]) + ' [Boundary]',
                                                        compartment='x'): 1})
                    continue
                t_model.add_reaction(rx)

        if data.equ != 'nan':
            t_model.add_reaction(data.equ)

        for gene_id in ["ENSG00000102780","ENSG00000197375"]:
            t_model.genes.get_by_id(gene_id).knock_out()

        if t_model.optimize().objective_value is None:
            res.append(0)
        else:
            res.append(1)

        t2 = model_list[3].metabolites.get_by_id('temp001s[s]').reactions
        for r in t2:
            print(r)
            print(r.flux)

        for r in t_model.boundary:
            if r.flux > 0:
                print('----------------')
                print(r)
                print(r.flux)
                print(r.bounds)



print(res)

C:\Users\Sigve\miniconda3\envs\Master\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


HMR_10023: temp001c[c] --> temp001s[s]
0.0
ess_57_out_1: temp001s[s] --> 
0.0
HMR_10024: temp001s[s] --> temp001x[x]
0.0
----------------
HMR_9813: m02642s[s] --> 
67.51510003775009
(0, 1000)
----------------
HMR_9036: m02389s[s] --> 
1000.0
(0, 1000)
----------------
HMR_9058: m01596s[s] --> 
682.7241411853529
(0, 1000)
----------------
HMR_9061: m01307s[s] --> 
324.3370139675349
(0, 1000)
----------------
HMR_9068: m02770s[s] --> 
168.6414684786712
(0, 1000)
----------------
HMR_9079: m02039s[s] --> 
1000.0
(0, 1000)
----------------
HMR_9086: m01252s[s] --> 
1080.5341638354093
(0, 1000)
----------------
HMR_9133: m02819s[s] --> 
74.79567761419435
(0, 1000)
----------------
HMR_9135: m02403s[s] --> 
925.2043223858057
(0, 1000)
----------------
HMR_9234: m01115s[s] --> 
257.04133635334097
(0, 1000)
----------------
HMR_9422: m02997s[s] --> 
742.958663646659
(0, 1000)
----------------
HMR_9437: m03118s[s] --> 
257.041336353341
(0, 1000)
----------------
EX_hiscyscys[e]: hiscyscys_s[s] 

In [2]:
print(model.constraints)



NameError: name 'model' is not defined